In [ ]:
!pip install scikit-learn timm pandas einops ipywidgets

In [ ]:
!pip install pybaselines

In [ ]:
!export HF_ENDPOINT=https://hf-mirror.com

In [1]:
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import timm
from timm.layers.classifier import ClassifierHead
import pandas as pd
from utils.datasets import ImageDataset
from sklearn.model_selection import train_test_split
from utils.misc import *
import os
import torch

from models.titration_model import CustomModule

In [3]:
def get_ds(root_path):
    ds = pd.read_csv(os.path.join(root_path, 'labels.csv'), dtype=str)
    num_classes = len(ds.iloc[:, 0].unique())

    # 划分训练集和验证集（根据整个video划分）
    valid_videos = ['MP_5', 'MP_17', 'VID_20250114_123113', 'VID_20250114_124017']
    test_videos = ['MM_3', 'MP_3', 'MP_19', 'VID_20250114_122658', 'VID_20250114_130314', 'video_20250114_123706']

    all_videos = ds.iloc[:, 1].unique()
    train_videos = list(set(all_videos) - set(valid_videos) - set(test_videos))

    X_train = ds[ds.iloc[:, 1].isin(train_videos)].reset_index(drop=True)
    X_valid = ds[ds.iloc[:, 1].isin(valid_videos)].reset_index(drop=True)
    X_test = ds[ds.iloc[:, 1].isin(test_videos)].reset_index(drop=True)

    train_ds = ImageDataset(X_train, root_path, transform)
    valid_ds = ImageDataset(X_valid, root_path, transform)
    test_ds = ImageDataset(X_test, root_path, transform)

    return num_classes, train_ds, valid_ds, test_ds

In [9]:
# 设置设备（GPU或CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

# 数据预处理
transform = transforms.Compose([
    transforms.CenterCrop(224),
    # transforms.RandomRotation(degrees=10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

root_path = r'./data/MR'

num_classes, train_ds, valid_ds, test_ds = get_ds(root_path)


# 数据预处理
test_transform = transforms.Compose([
    transforms.Resize(224),
    # transforms.RandomRotation(degrees=10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

valid_ds.set_transform(test_transform)
test_ds.set_transform(test_transform)
# 创建数据加载器
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=6, pin_memory=True)
valid_loader = DataLoader(valid_ds, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

# 实例化模型
model_name = 'convnext_tiny'
model_name = 'maxvit_tiny_rw_224'
model_name = 'swinv2_cr_small_ns_224'
model_name = 'resnetv2_50x1_bit'
model_name = 'resnest26d'
model_name = 'coat_tiny'
model_name = 'convit_tiny'

model = CustomModule(model_name, num_classes=num_classes)
# model.load_state_dict(torch.load('./outputs/checked/MR_convnext_tiny_20250202032022_991_990.pth'))

log_file = f"./logs/{model_name}-{datetime.now().strftime('%Y%m%d')}.log"
init_logger(log_file)
logging.info('-' * 120)

kwargs = {}
kwargs['model_name_prefix'] = f'MR_{model_name}'
kwargs['patience'] = 10
# 训练模型
best_model, results = train_predict(model, device, train_loader, valid_loader, test_loader, **kwargs)
# logging.info(results)


Loading pretrained weights from Hugging Face hub (timm/convit_tiny.fb_in1k)
[timm/convit_tiny.fb_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
Missing keys (head.weight, head.bias) discovered while loading pretrained weights. This is expected if model is being adapted.
------------------------------------------------------------------------------------------------------------------------
Epoch 0: 100%|██████████| 67/67 [00:02<00:00, 25.12batch/s]
epoch:0. Train Accuracy: 0.9095 Train Loss: 0.519163.
epoch:0. Valid Accuracy: 0.8135 Valid Loss: 0.723545. patience count: 0
Epoch 1: 100%|██████████| 67/67 [00:02<00:00, 24.19batch/s]
epoch:1. Train Accuracy: 0.9750 Train Loss: 0.405924.
epoch:1. Valid Accuracy: 0.8394 Valid Loss: 0.706168. patience count: 0
Epoch 3: 100%|██████████| 67/67 [00:02<00:00, 23.43batch/s]
epoch:3. Train Accuracy: 0.9840 Train Loss: 0.383692.
epoch:3. Valid Accuracy: 0.8652 Valid Loss: 0.65287

In [ ]:
import numpy as np
import torch
from fvcore.nn import FlopCountAnalysis
from fvcore.nn import flop_count_table

from timm.models import create_model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
new_model = results[0]

flops = FlopCountAnalysis(new_model, torch.rand(1, 3, 224, 224) .to(device))
print(flop_count_table(flops, max_depth=2))

In [ ]:
!unzip MR.zip -d ./data